# Data Mining

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import time
import unicodedata
from bs4 import BeautifulSoup
from uuid import uuid4

In [2]:
from dataclasses import dataclass

@dataclass
class Course:
    id: str
    title: str
    units: str
    description: str

In [3]:
depts_req = requests.get('https://catalog.ucsd.edu/front/courses.html')
depts_bs = BeautifulSoup(depts_req.text)

In [4]:
depts_elems = pd.Series(depts_bs.find_all('a', href=True)).map(lambda x: x['href'])
dept_course_links = (
    depts_elems[depts_elems.str.contains('/courses/')]
    .str.lstrip('../')
    .map(lambda x: 'https://catalog.ucsd.edu/' + x)
)

In [5]:
depts_bs = {}
for dept_url in dept_course_links:
    time.sleep(2)
    dept = dept_url.split('/')[-1].rstrip('.html')
    print('Requesting %s Courses' % dept)
    dept_page = requests.get(dept_url)
    dept_bs = BeautifulSoup(dept_page.text)
    depts_bs[dept] = dept_bs

Requesting AIP Courses
Requesting AASM Courses
Requesting AWP Courses
Requesting ANTH Courses
Requesting AUDL Courses
Requesting BIOI Courses
Requesting BIOL Courses
Requesting BIOM Courses
Requesting CHEM Courses
Requesting CHIN Courses
Requesting CLAS Courses
Requesting CCS Courses
Requesting CSP Courses
Requesting CLIN Courses
Requesting CLRE Courses
Requesting COGS Courses
Requesting COMM Courses
Requesting CONT Courses
Requesting CGS Courses
Requesting CAT Courses
Requesting DSC Courses
Requesting DSGN Courses
Requesting DOC Courses
Requesting ECON Courses
Requesting EDS Courses
Requesting ERC Courses
Requesting ENG Courses
Requesting BENG Courses
Requesting CSE Courses
Requesting CSE-AESE Courses
Requesting ECE Courses
Requesting MAE Courses
Requesting NANO Courses
Requesting SE Courses
Requesting ENVR Courses
Requesting ESYS Courses
Requesting ETHN Courses
Requesting FILM Courses
Requesting GLBH Courses
Requesting GPS Courses
Requesting GSS Courses
Requesting LHCO Courses
Reques

In [8]:
def extract_course_info(heading_elem):
    course_heading_text = unicodedata.normalize('NFKD', heading_elem.text)

    course_id_regex = re.search('[(]?\w+[)]? ((\d+\w*)(, )?)+', course_heading_text)
    if course_id_regex is None:
        print("Warning! Did not find course id in %s" % course_heading_text)
        return None
    course_id = course_id_regex.group(0).replace('(', '').replace(')', '')

    course_units_regex = re.search('(\d)', course_heading_text[course_id_regex.end(0):])
    if course_units_regex is not None:
        course_units = course_units_regex.group(0)
        course_title = course_heading_text[course_id_regex.end(0)+2:course_id_regex.end(0)+course_units_regex.start(0)-2]
    else:
        course_units = None
        course_title = course_heading_text[course_id_regex.end(0)+2:]

    course_description_elem = heading_elem.find_next_sibling('p', 'course-descriptions')
    if course_description_elem is not None:
        course_description_text = unicodedata.normalize('NFKD', course_description_elem.text)
    else:
        course_description_text = None

    return Course(
        id=course_id,
        title=course_title,
        units=course_units,
        description=course_description_text,
    )

In [9]:
courses = []
for dept, bs in depts_bs.items():
    print(dept)
    anchors = bs.find_all('p', 'course-name')
    for anchor in anchors:
        course = extract_course_info(anchor)
        if course is not None:
            courses.append(course)

df = pd.DataFrame(courses).drop_duplicates()

AIP
AASM
AWP
ANTH
AUDL
BIOI
BIOL
BIOM
CHEM
CHIN
CLAS
CCS
CSP
CLIN
CLRE
COGS
COMM
CONT
CGS
CAT
DSC
DSGN
DOC
ECON
EDS
ERC
ENG
BENG
CSE
CSE-AESE
ECE
MAE
NANO
SE
ENVR
ESYS
ETHN
FILM
GLBH
GPS
GSS
LHCO
HIST
HDS
HMNR
HUM
INTL
JAPN
JWSP
LATI
LAWS
LING
LIT
MMW
MGT
MBC
Warning! Did not find course id in Electives. Varies (12)
MATS
MATH
MSED
MCWP
MUS
NEU
PHIL
PHYS
POLI
PSYC
FMPH
RELI
REV
SCIS
SIO
SEV
SXTH
SOC
SYN
THEA
TMC
USP
VIS
WARR


In [10]:
df.sample(5)

,id,title,units,description
6277,TDGR 280C,Stage Managemen,3,The final course in the three-part introductor...
952,COGS 201,Neural Dynamics of Cognition,4,This course surveys the development of symboli...
3046,HIEU 127,Sport in the Modern World,4,This course looks at the phenomenon of sport i...
3163,HINE 136GS,Zionism and Post Zionism,4,This course explores the evolution of Zionism ...
1830,CSE 218,Advanced Topics in Software Engineering,4,This course will cover a current topic in soft...


## Cleaning

In [12]:
@dataclass
class CoursePreqs:
    course_id: str
    upper_div_only: bool
    grad_only: bool
    prereqs: set()

@dataclass(frozen=True)
class Preq:
    id: uuid4
    courses: frozenset
    n_required = 1

In [39]:
def parse_preqs(course_id, description):
    if description is None or 'Prerequisites:' not in description:
        return None

    prereqs = description.split('Prerequisites:')[1].strip()
    upper_div_only = 'upper-division standing' in prereqs
    grad_only = 'graduate' in prereqs

    depts = re.finditer('[A-Z]{2,4}', prereqs)
    courses = re.finditer('(?<=[ ])\d{1,3}\w*', prereqs)

    depts_df = pd.DataFrame([(x.group(0), x.start(0), x.end(0)) for x in depts], columns=['dept', 'start', 'end'])
    courses_df = pd.DataFrame([(x.group(0), x.start(0), x.end(0)) for x in courses], columns=['course', 'start', 'end'])

    def calc_course_join(course_start):
        for i, dept in depts_df.iterrows():
            if dept.start > course_start:
                return depts_df.iloc[i-1].dept
        return depts_df.iloc[-1].dept

    prereq_courses = prereqs
    try:
        courses_df = courses_df.assign(dept = courses_df.start.map(calc_course_join))
    except:
        print('Failed to assign courses: %s' % course_id)
        return None

    for idx in reversed(courses_df.index):
        course = courses_df.iloc[idx]
        course_name = "%s %s" % (course.dept, course.course)
        prereq_courses = prereq_courses[:course.start] + course_name + prereq_courses[course.end:]

    prereq_courses = prereq_courses.split(' ')
    prereq_courses = ' '.join([x for i, x in enumerate(prereq_courses) if i == len(prereq_courses) - 1 or prereq_courses[i] != prereq_courses[i+1]])

    prereq_course_set = set()
    for section in prereq_courses.split('and'):
        section = section.strip()
        courses = [x for x in re.findall('\w{2,4} \d+\w*', section) if not x.startswith('GPA')]
        if len(courses) == 0:
            continue
        courses = Preq(
            id = str(uuid4()),
            courses = frozenset(courses),
        )
        prereq_course_set.add(courses)
    
    return CoursePreqs(
        course_id = course_id,
        upper_div_only = upper_div_only,
        grad_only = grad_only,
        prereqs = prereq_course_set
    )

In [40]:
preqs = df.apply(lambda x: parse_preqs(x.id, x.description), axis=1);

Failed to assign courses: COGS 190D
Failed to assign courses: BENG 202
Failed to assign courses: CSE 282
Failed to assign courses: HIGR 215A
Failed to assign courses: HIGR 216A
Failed to assign courses: HIGR 230A
Failed to assign courses: HIGR 267A
Failed to assign courses: LIFR 5B, 5C, 5D
Failed to assign courses: LIGM 5A, 5B, 5C, 5D
Failed to assign courses: LISP 5A, 5B, 5C, 5D
Failed to assign courses: MATH 104C
Failed to assign courses: MATH 241A
Failed to assign courses: NEU 426
Failed to assign courses: NEU 427
Failed to assign courses: POLI 247C
Failed to assign courses: POLI 271
Failed to assign courses: POLI 287
Failed to assign courses: POLI 288
Failed to assign courses: POLI 289


In [41]:
parsed_preqs = pd.DataFrame(
    preqs.apply(lambda x: (x.course_id, x.upper_div_only, x.grad_only, [y.id for y in x.prereqs]) if x is not None else None).to_list(),
    columns=['id', 'upper_only', 'grad_only', 'prerequisites_id']
)
courses_merged = df.merge(parsed_preqs, on='id')

In [42]:
courses_merged

,id,title,units,description,upper_only,grad_only,prerequisites_id
0,AIP 97,Academic Internship,2,Individual placements for field learning. Must...,False,True,[]
1,AIP 197,Academic Internship Program,2,Individual internship placements integrated wi...,True,False,[]
2,AIP 197DC,"UCDC: Washington, DC Internship",6,This internship is attached to the University ...,True,False,[]
3,AIP 197F,Academic Internship Program—Foreign,4,Individual placements for field learning oppor...,False,False,[]
4,AIP 197P,Public Service Internship,4,Individual placements for field learning assoc...,False,False,[]
...,...,...,...,...,...,...,...
5032,WCWP 10B,The Writing Course B,4,A seminar-style course in reading and writing ...,False,False,[2d9db568-7b0a-4a4e-9423-8e789c41545f]
5033,WCWP 100,Academic Writing,4,An upper-division workshop course in argumenta...,False,False,[]
5034,WCWP 160,Technical Writing for Scientists and Engineers,4,An upper-division workshop-style writing cours...,False,False,[]
5035,WARR 189,Academic Mentoring and the Writing Process,2,Students will gain a fundamental understanding...,False,False,[]


In [43]:
courses_cleaned = courses_merged.drop('prerequisites_id', axis=1)
courses_prerequisites = courses_merged[['id', 'prerequisites_id']].explode('prerequisites_id').dropna()

In [44]:
prereq_groups = preqs.apply(lambda x: list(x.prereqs) if x is not None else None).explode().dropna()
prerequisites = pd.DataFrame(prereq_groups.apply(lambda x: (x.id, x.n_required, x.courses)).to_list(), columns=['group_id', 'n_required', 'courses'])
prerequisites_cleaned = (
    prerequisites
    .assign(courses = prerequisites.courses.apply(lambda x: list(x)))
    .explode('courses')
)

prerequisites_cleaned = prerequisites_cleaned.assign(id = [str(uuid4()) for x in range(prerequisites_cleaned.shape[0])])
prerequisites_cleaned

,group_id,n_required,courses,id
0,4a995373-1597-4338-aab4-f32dc2d9230b,1,AWP 4A,4ec4d91e-187c-4474-abc4-cb9c9e99590a
1,9b5dfa96-8bf7-4d1f-b9da-86a52f3ac5d5,1,ANTH 196A,38df2bd9-fc4d-4ea0-995a-5c740a67e7c5
2,464500ce-4256-49cc-bf65-9556d36a9edd,1,ANTH 196A,91c0dff5-88fe-492d-b25f-d953af77fefc
3,629e0687-a350-40eb-9410-33ef9e721eff,1,ANPR 198,4d842e8d-d778-4bf4-96e8-309b8efc5228
4,204bfa72-20a2-441d-affe-a3396c242545,1,ANPR 198,f4184707-093d-4852-8d50-ca3cd93bc750
...,...,...,...,...
3125,b29d126d-9903-4c8e-b981-0bd3e1995278,1,VIS 145A,01a9aaa1-1306-4ba0-8f4c-77af691d72f6
3126,7e94c8cd-6f2a-4d0e-a449-18cbd7cb20db,1,VIS 142,aa5d1006-8300-46be-9ab6-1bfa33454c8d
3126,7e94c8cd-6f2a-4d0e-a449-18cbd7cb20db,1,VIS 135,99e6be1a-785c-468a-826b-9f7b14354ad1
3127,c78ebd7c-1296-46b3-a529-29885c297991,1,VIS 84,40d3c25e-0828-4379-9883-7c892cfa2cf1


## Saving to DB

In [6]:
from sqlalchemy import create_engine

In [7]:
engine = create_engine('<DB>')

In [27]:
courses_cleaned.to_sql('courses', engine, method='multi')

In [45]:
courses_prerequisites

,id,prerequisites_id
10,AWP 4B,4a995373-1597-4338-aab4-f32dc2d9230b
32,ANTH 196B,9b5dfa96-8bf7-4d1f-b9da-86a52f3ac5d5
33,ANTH 196C,464500ce-4256-49cc-bf65-9556d36a9edd
34,ANTH 197,629e0687-a350-40eb-9410-33ef9e721eff
35,ANTH 198,204bfa72-20a2-441d-affe-a3396c242545
...,...,...
5011,VIS 185,80fb4bf4-a8ff-4493-be42-651ba51bd8bb
5012,VIS 190,b29d126d-9903-4c8e-b981-0bd3e1995278
5012,VIS 190,7e94c8cd-6f2a-4d0e-a449-18cbd7cb20db
5013,VIS 194S,c78ebd7c-1296-46b3-a529-29885c297991


In [46]:
courses_prerequisites.to_sql('courses_prerequisites', engine, method='multi', if_exists='replace')

In [47]:
prerequisites_cleaned.to_sql('prerequisites', engine, method='multi', if_exists='replace')